### HARVARD'S ARTIFACTS COLLECTION : ETL,ANALYTICS AND STREMLIT AND SHOWCASE

In [1]:
!pip install requests

### IMPORT REQUESTS

In [2]:
# import requests 
#url=classification

import requests
api_key="2b36e286-db1f-4d5c-adbc-afb0ec1a8d96"
url="https://api.harvardartmuseums.org/classification"
params={
    "apikey":api_key,
    "size":100
}
response=requests.get(url,params)
response

<Response [200]>

In [3]:
data=response.json()

In [4]:
data

{'info': {'totalrecordsperquery': 100,
  'totalrecords': 64,
  'pages': 1,
  'page': 1,
  'responsetime': '5 ms'},
 'records': [{'objectcount': 1,
   'name': 'Natural History Specimens',
   'id': 1133,
   'lastupdate': '2025-08-29T04:32:58-04:00',
   'classificationid': 1133},
  {'objectcount': 1,
   'name': 'Paintings with Text',
   'id': 197,
   'lastupdate': '2025-08-29T04:32:58-04:00',
   'classificationid': 197},
  {'objectcount': 66,
   'name': 'Performance Artifacts',
   'id': 224,
   'lastupdate': '2025-08-29T04:32:58-04:00',
   'classificationid': 224},
  {'objectcount': 470,
   'name': 'Rubbings',
   'id': 173,
   'lastupdate': '2025-08-29T04:32:58-04:00',
   'classificationid': 173},
  {'objectcount': 62,
   'name': 'Brick Stamps',
   'id': 304,
   'lastupdate': '2025-08-29T04:32:58-04:00',
   'classificationid': 304},
  {'objectcount': 46,
   'name': 'Cameos',
   'id': 1086,
   'lastupdate': '2025-08-29T04:32:58-04:00',
   'classificationid': 1086},
  {'objectcount': 1,
   

In [5]:
type(data)

dict

In [6]:
data.keys()

dict_keys(['info', 'records'])

### FETCH ONLY RECORDS

In [7]:
data["records"]

[{'objectcount': 1,
  'name': 'Natural History Specimens',
  'id': 1133,
  'lastupdate': '2025-08-29T04:32:58-04:00',
  'classificationid': 1133},
 {'objectcount': 1,
  'name': 'Paintings with Text',
  'id': 197,
  'lastupdate': '2025-08-29T04:32:58-04:00',
  'classificationid': 197},
 {'objectcount': 66,
  'name': 'Performance Artifacts',
  'id': 224,
  'lastupdate': '2025-08-29T04:32:58-04:00',
  'classificationid': 224},
 {'objectcount': 470,
  'name': 'Rubbings',
  'id': 173,
  'lastupdate': '2025-08-29T04:32:58-04:00',
  'classificationid': 173},
 {'objectcount': 62,
  'name': 'Brick Stamps',
  'id': 304,
  'lastupdate': '2025-08-29T04:32:58-04:00',
  'classificationid': 304},
 {'objectcount': 46,
  'name': 'Cameos',
  'id': 1086,
  'lastupdate': '2025-08-29T04:32:58-04:00',
  'classificationid': 1086},
 {'objectcount': 1,
  'name': 'Casts',
  'id': 1139,
  'lastupdate': '2025-08-29T04:32:58-04:00',
  'classificationid': 1139},
 {'objectcount': 6,
  'name': 'Containers',
  'id': 1

In [8]:
type(data["records"])

list

In [9]:
!pip install streamlit streamlit-option-menu

###  IMPORTS AND CONSTANTS

In [10]:
# IMPORT REQUESTS,SQLITE3,PANDAS,STREAMLIT,TIME

import requests
import sqlite3
import pandas as pd
import streamlit as st
import time

API_KEY = "2b36e286-db1f-4d5c-adbc-afb0ec1a8d96"
BASE_URL = "https://api.harvardartmuseums.org/object"
CLASS_URL = "https://api.harvardartmuseums.org/classification"

### FETCH DATA FOR A CLASSIFICATION

In [11]:
# FETCH RECORDS FOR A CLASSIFICATION

def fetch_data(classification, max_records=2500):
    """Fetch up to max_records for given classification (synchronous)."""
    records = []
    page = 1
    page_size = 100
    while len(records) < max_records:
        params = {
            "apikey": API_KEY,
            "size": page_size,
            "page": page,
            "classification": classification
        }
        try:
            r = requests.get(BASE_URL, params=params, timeout=30)
            r.raise_for_status()
            data = r.json()
        except Exception as e:
            st.error(f"Network/API error: {e}")
            break

        page_records = data.get("records", [])
        if not page_records:
            break
        records.extend(page_records)
         
        if len(page_records) < page_size:
            break
        page += 1
        time.sleep(0.1)  # polite
    return records[:max_records]

### SPLIT RECORDS INTO METADATA,MEDIA,COLORS

In [12]:
# SPLIT DATA INTO 3 LISTS

def artifacts_details(records):
    """Extract metadata, media, colors lists from raw API records."""
    metadata, media, colors = [], [], []
    for r in records:
        # metadata
        metadata.append({
            "id": r.get("id"),
            "title": r.get("title"),
            "culture": r.get("culture"),
            "period": r.get("period"),
            "century": r.get("century"),
            "medium": r.get("medium"),
            "dimensions": r.get("dimensions"),
            "description": r.get("description"),
            "department": r.get("department"),
            "classification": r.get("classification"),
            "accessionyear": r.get("accessionyear"),
            "accessionmethod": r.get("accessionmethod")
        })
        objid = r.get("id")  # <-- use record id as objectid to link tables
        # media
        media.append({
            "objectid": objid,
            "imagecount": r.get("imagecount"),
            "mediacount": r.get("mediacount"),
            "colorcount": r.get("colorcount"),
            "rank_num": r.get("rank"),
            "datebegin": r.get("datebegin"),
            "dateend": r.get("dateend")
        })
        # colors (list)
        colors_list = r.get("colors")
        if isinstance(colors_list, list):
            for c in colors_list:
                colors.append({
                    "objectid": objid,
                    "color": c.get("color"),
                    "spectrum": c.get("spectrum"),
                    "hue": c.get("hue"),
                    "percent": c.get("percent"),
                    "css3": c.get("css3")
                })
    return metadata, media, colors

### CREATE DATABASE TABLES

In [13]:
# CREATE DATABASE TABLES
    
def create_tables(conn):
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS artifacts_metadata(
            id INTEGER PRIMARY KEY,
            title TEXT,
            culture TEXT,
            period TEXT,
            century TEXT,
            medium TEXT,
            dimensions VARCHAR(75),
            description TEXT,
            department TEXT,
            classification TEXT,
            accessionyear INTEGER,
            accessionmethod TEXT
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS artifacts_media(
            objectid INTEGER,
            imagecount INTEGER,
            mediacount INTEGER,
            colorcount INTEGER,
            rank_num INTEGER,
            datebegin INTEGER,
            dateend INTEGER,
            FOREIGN KEY(objectid) REFERENCES artifacts_metadata(id)
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS artifacts_colors(
            objectid INTEGER,
            color TEXT,
            spectrum TEXT,
            hue TEXT,
            percent REAL,
            css3 TEXT,
            FOREIGN KEY(objectid) REFERENCES artifacts_metadata(id)
        )
    """)
    conn.commit()


### INSERT VALUES INTO DATABASE

In [14]:
# INSERT VALUES INTO DATABASE
    
def insert_values(conn, metadata, media, colors):
    cur = conn.cursor()
    insert_metadata = """
        INSERT OR IGNORE INTO artifacts_metadata
        (id,title,culture,period,century,medium,dimensions,description,department,classification,accessionyear,accessionmethod)
        VALUES (?,?,?,?,?,?,?,?,?,?,?,?)
    """
    insert_media = """
        INSERT INTO artifacts_media
        (objectid,imagecount,mediacount,colorcount,rank_num,datebegin,dateend)
        VALUES (?,?,?,?,?,?,?)
    """
    insert_colors = """
        INSERT INTO artifacts_colors
        (objectid,color,spectrum,hue,percent,css3)
        VALUES (?,?,?,?,?,?)
    """

# prepare tuples (use .get in case of missing keys)
    meta_tuples = [
        (m.get("id"), m.get("title"), m.get("culture"), m.get("period"), m.get("century"),
         m.get("medium"), m.get("dimensions"), m.get("description"), m.get("department"),
         m.get("classification"), m.get("accessionyear"), m.get("accessionmethod"))
        for m in metadata
    ]
    media_tuples = [
        (m.get("objectid"), m.get("imagecount"), m.get("mediacount"), m.get("colorcount"),
         m.get("rank_num"), m.get("datebegin"), m.get("dateend"))
        for m in media
    ]
    colors_tuples = [
        (c.get("objectid"), c.get("color"), c.get("spectrum"), c.get("hue"), c.get("percent"), c.get("css3"))
        for c in colors
    ]

